## 3. DEMO CODE

### 3.1 Introduction

We load the best classifier computed in the previous notebook (2ClassifierTrainning.ipynb) and we use it on our demo data. Also, the user can select a random image and gets the result of the prediction back.

In [ ]:
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
import glob
import os
import pandas as pd
import cv2
from skimage.filters import sobel
from skimage.feature import hog
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import time
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import joblib
import urllib.request
import validators

### 3.2 Load All relevant Functions

In [ ]:
def createRocCurve(y_scores, y_onehot):

    np.random.seed(0)

    # Create an empty figure, and iteratively add new lines
    # every time we compute a new class
    fig = go.Figure()
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    for i in range(y_scores.shape[1]):
        y_true = y_onehot.iloc[:, i]
        y_score = y_scores[:, i]

        fpr, tpr, _ = roc_curve(y_true, y_score)
        auc_score = roc_auc_score(y_true, y_score)

        name = f"{y_onehot.columns[i]} (AUC={auc_score:.2f})"
        fig.add_trace(go.Scatter(x=fpr, y=tpr, name=name, mode='lines'))

    fig.update_layout(
        xaxis_title='False Positive Rate',
        yaxis_title='True Positive Rate',
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=700, height=500
    )
    
    return fig

### Confusion Matric function

def conf_m(cm, classes_dict):
    z = cm

    # invert z idx values
    z = z[::-1]

    #x = ['healthy', 'multiple diseases', 'rust', 'scab']
    x = classes_dict
    y =  x[::-1].copy() # invert idx values of x

    # change each element of z to type string for annotations
    z_text = [[str(y) for y in x] for x in z]

    # set up figure 
    fig = ff.create_annotated_heatmap(z, x=x, y=y, annotation_text=z_text, colorscale='Viridis')

    # add title
    #fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
    #                  #xaxis = dict(title='x'),
    #                  #yaxis = dict(title='x')
    #                 )

    # add custom xaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=0.5,
                            y=-0.15,
                            showarrow=False,
                            text="Predicted value",
                            xref="paper",
                            yref="paper"))

    # add custom yaxis title
    fig.add_annotation(dict(font=dict(color="black",size=14),
                            x=-0.35,
                            y=0.5,
                            showarrow=False,
                            text="Real value",
                            textangle=-90,
                            xref="paper",
                            yref="paper"))

    # adjust margins to make room for yaxis title
    fig.update_layout(
        margin=dict(t=50, l=200),
        width=700, height=500
        )

    # add colorbar
    fig['data'][0]['showscale'] = True
    #fig.show()
    return fig


# Precission REcall Curves

def pr_rec_curve(y_onehot, y_scores):

    np.random.seed(0)

    # Create an empty figure, and iteratively add new lines
    # every time we compute a new class
    fig = go.Figure()
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=1, y1=0
    )

    for i in range(y_scores.shape[1]):
        y_true = y_onehot.iloc[:, i]
        y_score = y_scores[:, i]

        precision, recall, _ = precision_recall_curve(y_true, y_score)
        auc_score = average_precision_score(y_true, y_score)

        name = f"{y_onehot.columns[i]} (AP={auc_score:.2f})"
        fig.add_trace(go.Scatter(x=recall, y=precision, name=name, mode='lines'))

    fig.update_layout(
        xaxis_title='Recall',
        yaxis_title='Precision',
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis=dict(constrain='domain'),
        width=700, height=500
    )
    
    return fig

### evaluate model function, prints accuracy and error
def evaluate_model(model, test_labels):
    start = time.time()
    prediction = model.predict(X_test)
    stop = time.time() 
    print(f"Total inference time: {round(stop - start, 2)}s")
    print(f"Inference time per example: {round((stop - start)/len(y_test),5)}s")
    print(f"Test Set Accuracy : {accuracy_score(y_test, prediction) * 100} %\n\n")
    
    return accuracy_score(y_test, prediction) * 100

def display_metrics(model, X_test, y_test):
    prediction = model.predict(X_test)
    y_scores = model.predict_proba(X_test)
    y_onehot = pd.get_dummies(y_test, columns=model.classes_)
    y_onehot = pd.get_dummies(y_test, columns=model.classes_)
    pred_score = round(accuracy_score(y_test, prediction) * 100.0, 2)
    cm = confusion_matrix(y_test, prediction, labels=model.classes_)
    
    print(classification_report(y_test, prediction))
    #print(type(classification_report(y_test, prediction)))

    conf_matrix = conf_m(cm, list(model.classes_))
    conf_matrix.show()
    roc_figure = createRocCurve(y_scores, y_onehot)
    roc_figure.show()
    prec_recall = pr_rec_curve(y_onehot, y_scores)
    prec_recall.show()


def plot_pca(feature_matrix_pca):
    fig = px.scatter(feature_matrix_pca, x="pca_1", y="pca_2", color="painter",
                 hover_data=['painter'])
    return fig

### 3.2.1 Demo Tools Function
In this segment we define a method that is used for the final demo production. This method takes as input an image url and it outputs a prediction of the image's creator based on a trained model that we have created.

In [ ]:
def demo_predictor(model, img_url):
    if model == 'RandomForest':
        demo_clf = joblib.load('best_rf_clf.joblib')
    elif model == 'SVM':
        demo_clf = joblib.load('best_svm_clf.joblib')
    else:
        demo_clf = joblib.load('best_knn_clf.joblib')
        
    x_train1 = img_process(img_url)
    x_train1 = x_train1 / 255.0

    image_features1, hog_size1 = feature_extractor(x_train1)
    pd_single_img1 = create_feature_matrix(x_train1, image_features1)
    
    X_test_single = pd_single_img1.values
    probability_demo=demo_clf.predict_proba(X_test_single)

    Categories = list(demo_clf.classes_)
    print(demo_clf.predict(X_test_single))
    results = []
    for ind,val in enumerate(Categories):
        print(f'{val} = {probability_demo[0][ind]*100}%')
        results.append(str(ind) + str(val))
       # results.append(val)
    return results

def img_process(img_path):
    SIZE = 64

    img_array = []
    
    valid=validators.url(img_path)
    
    if valid==True:
        req = urllib.request.urlopen(img_path)
        arr = np.asarray(bytearray(req.read()), dtype=np.uint8)
        img = cv2.imdecode(arr, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE,SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_array.append(img)
        img_array = np.array(img_array)
    else:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE,SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_array.append(img)
        img_array = np.array(img_array)

    return img_array

# feature extraction function
def feature_extractor(dataset):
    SIZE = 64
    image_dataset = pd.DataFrame()
    for image in range(dataset.shape[0]):
        df = pd.DataFrame()
        input_img = dataset[image, :, :]  # one more dimension [:] if I have color
        img = input_img
        
        # Pixel values
        pixel_values = img.reshape(-1)
        df['Pixel_Value'] = pixel_values
        #df['Image_Name'] = image
        
        # Gabor
        num = 1
        kernels = []
        for theta in range(2):
            theta = theta / 4. * np.pi
            for sigma in range(1,3):  # range(1,3) default
                lamda = np.pi / 4.
                gamma = 0.5
                gabor_label = 'Gabor' + str(num)
                ksize = 9
                kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, 0, ktype=cv2.CV_32F)    
                kernels.append(kernel)
                
                fimg = cv2.filter2D(img, cv2.CV_8UC3, kernel)
                filtered_img = fimg.reshape(-1)
                df[gabor_label] = filtered_img  #Labels columns as Gabor1, Gabor2, etc.
#                print(gabor_label, ': theta=', theta, ': sigma=', sigma, ': lamda=', lamda, ': gamma=', gamma)
                num += 1  #Increment for gabor column label
    
        # Hog
        resized_img = cv2.resize(img, (64,128))
        fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8), 
                    cells_per_block=(2, 2), visualize=True)
        
        hog_size = fd.shape
        d_zeroes = pd.DataFrame(np.zeros((SIZE*SIZE, 1)))
        d_hog = pd.DataFrame(fd)
        df2 = d_zeroes.iloc[3780:,:]
        df3 = pd.concat([d_hog,df2],axis=0, ignore_index=True)
        df['hog'] = df3

            
        image_dataset = pd.concat([image_dataset, df],axis=0) 
    return image_dataset, hog_size

### 3.3 Demo Data Loading and PCA Displaying

In [ ]:
### Load Feature Matrix from .csv file
feature_matrix_demo = pd.read_csv('feature_matrix_demo.csv')
feature_matrix_demo_pca = pd.read_csv('feature_matrix_demo_pca.csv')

# Display the pca reduced image of the dataset:
fig = px.scatter_3d(feature_matrix_demo_pca, x='pca_1', y='pca_2', z='pca_3',
              color='painter')
fig.show()

### 3.4 Classifier Loading and fitting on DEMO DATA

In [ ]:
# Set basic data variables
X_test_demo = feature_matrix_demo.drop(columns = 'painter')
X_test_demo.values
y_test_demo = feature_matrix_demo['painter']
y_test_demo = y_test_demo.values

In [ ]:
# load one of the classsifiers trained and optimized before and get predictions

demo_clf = joblib.load('best_rf_clf.joblib')
#demo_clf = joblib.load('best_svm_clf.joblib')
#demo_clf = joblib.load('best_knn_clf.joblib')

prediction_demo=demo_clf.predict(X_test_demo)
acc = accuracy_score(y_test_demo, prediction_demo)

# Display all the relevant metrics
#print((classification_report(y_test_demo, prediction_demo)))
#print(acc)

display_metrics(demo_clf, X_test_demo, y_test_demo)

## 3.5 Classifier URL Tool

This small segment of code calls the method demo_predictor which takes as an input a url pointing to an image, and return a prediction.

In [ ]:
url=input('Enter URL of Image')
# https://uploads0.wikiart.org/00129/images/claude-monet/impression-sunrise.jpg!Large.jpg
results = demo_predictor('RandomForest', url)   
print(results)